<a href="https://colab.research.google.com/github/Gaurav7004/RSE_TOOL/blob/main/TEST_PPSWR_only_6thSEPT2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import re
import pandas as pd
import numpy as np
import itertools

In [2]:
df = pd.read_csv('EX2_PPSWR.csv')

# remove blank spaces from front and back of column names
df.columns = df.columns.str.strip()
df.columns

<ipython-input-2-9914145c0ec0>:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('EX2_PPSWR.csv')


Index(['SEC', 'ST', 'AST', 'DC', 'NSSR', 'STRM', 'SSTRM', 'FSU', 'SSS', 'SSU',
       'CAPZS', 'SMLZS', 'DSI', 'NFSU', 'NSS', 'MULT', 'CAPH', 'SMLH', 'SRL',
       'HHSIZE', 'RELIGION', 'SG', 'MPCE_ALL', 'MPCE_ST', 'REL', 'SEX', 'AGE',
       'MARST', 'GEDU', 'TEDU'],
      dtype='object')

In [3]:
## Data is the dictionary
data = {}

## Keys
key = ''

## values
values = []

## Open
with open('Example2-PPSWR.RSE', "r") as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            if key:
                key = key.split("'")[0]
                key = key.split("\t")[0]
                data[key] = [v for v in values if v and not v.startswith("'")]
            key = line.split("'")[0]
            key = key.split("\t")[0]
            values = []
        elif line.startswith("'"):
            pass
        else:
            line = line.split("'")[0]
            values.append(line.strip())

# add last group
if key:
    data[key] = [v for v in values if v and not v.startswith("'")]

print(json.dumps(data, indent=4))

{
    "#SAMPLING PPSWR-SSS": [],
    "#GROUP 2": [
        "SEC   C sec",
        "AGE_GR C NEWVAR"
    ],
    "#VARIABLE 4  ": [
        "YP\tN  NEWVAR",
        "YM\tN  NEWVAR",
        "YF\tN  NEWVAR",
        "YH\tN  NEWVAR"
    ],
    "#RENAME  5    ": [
        "STRMID C  sec+st+strm+sstrm",
        "FSU    C  fsu",
        "SSS    C  sss",
        "USU    C  ssu",
        "NFSU   C  nfsu",
        "NSS    N  nss",
        "MULT   N  mult"
    ],
    "#TRANSFORM 5  ": [
        "AGE_GR=(\"01\") AGE<5",
        "AGE_GR=(\"02\") AGE>=5 && AGE<10",
        "AGE_GR=(\"03\") AGE>=10 && AGE<20",
        "AGE_GR=(\"04\") AGE>=20 && AGE<60",
        "AGE_GR=(\"05\") AGE>=60",
        "YP=(1)",
        "YM=(1)        SEX<>2",
        "YM=(0)\t\tOtherwise",
        "YF=(1)        SEX=2",
        "YF=(0)\t\tOtherwise",
        "YH=(1)        GEDU>=\"11\"",
        "YH=(0)\t\tOtherwise"
    ],
    "#EST_RSE 7": [
        "S YP",
        "E 1000*YF/YM",
        "R YF/YM",
        "R YH",
    

In [4]:
def SAMPLING_(data, df):
    for key, value in data.items():
        ## To read the sampling method used
        if str('SAMPLING').lower() in str(key).lower():
            if str('SUBSAMPLE2').lower() in str(key).lower():
                return str('SUBSAMPLE2')
            elif (str('PPSWR-SSS').lower() or str('PPSWR').lower()) in str(key).lower():
                return str('PPSWR-SSS')
            elif (str('SRSWR-SSS').lower() or str('SRSWR').lower()) in str(key).lower():
                return str('SRSWR-SSS')

SAMPLING_(data, df)

'PPSWR-SSS'

In [5]:
### ---------------------------------------------------------------------------------------------------------
def GROUP_(data, df):
    for key, value in data.items():
        if str('GROUP').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            for name in value:
                if name in df.columns:
                    pass
                elif name not in df.columns:
                    df[name] = 0

            return value


    for key, value in data.items():
        print(key, value)
        ### To read the group columns
        if str('GROUP').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\w+\s+\w+\s+\w+'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst = match.group().split()

                print(lst[0], lst[1])

                if lst[0] in df.columns:
                    if lst[1] == 'C':
                        df[lst[0]] = df[lst[0]].astype(str)
                elif lst[0] not in df.columns:
                    # if lst[1] == 'C':
                    #     df[lst[0]] = ''
                    pass

        else:
            pass

### ---------------------------------------------------------------------------------------------------------
def FILTER_(data, df):
    lst_queries = []
    for key, value in data.items():

        if str('FILTER').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\([^()]+\)|\w+[<>!=]+[0-9]+(?:\s*(?:&&|\|\|)\s*(?:\([^()]+\)|\w+[<>!=]+[0-9]+))*'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst_queries.append(match.group())

        else:
            pass

    filter_query = ' & '.join(['({})'.format(item) for item in lst_queries])

    try:
        df = df.query(filter_query)
        return df
    except:
        return df


### ---------------------------------------------------------------------------------------------------------
def FILE_(data):
    for key, value in data.items():
        ### To read the file used
        if str('FILE').lower() in str(key).lower():
            if key.strip().endswith('.xlsx') or key.strip().endswith('.csv'):
                match = re.search(r'[A-Za-z0-9]+\.xlsx|[A-Za-z0-9]+\.csv', key)
                if match:
                    file_name = match.group(0)
                    return file_name
                else:
                    file_name = None
                    return file_name
                print(file_name)

            # return file_name

### ---------------------------------------------------------------------------------------------------------
def RENAME_(data, df):
    # create an empty dictionary to store the column names
    column_dict = {}

    for key, value in data.items():
        ### To read the file used
        if str('RENAME').lower() in str(key).lower():
            # value = [elem for elem in value[0].split(' ') if elem.strip()]
            # return value

            for col in value:
                col_split = col.split()

                # print(col_split[0])

                # extract the last element as the column name
                col_name = col_split[-1]

                new_col_name = col_split[0]

                ### if there is a "+" sign in the last element, concatenate the columns
                if "+" in col_name:
                    col_concat = col_name.split("+")
                    col_concat = [i.upper() for i in col_concat]

                    # concatenate the columns
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if int(i) < 10 else str(i) for i in x]), axis=1)
                    print(df[new_col_name])

                else:
                    col_name = col_name.upper()

### ---------------------------------------------------------------------------------------------------------
def NEWVARIABLE_(data, df):
    for key, value in data.items():
        ### To read the file used
        if str('VARIABLE').lower() in str(key).lower():
            value = [s.split()[0] for s in value]

            for name in value:
                df[name] = 1

### ---------------------------------------------------------------------------------------------------------
def TRANSFORM_(data):
    global condition

    # Initialize empty arrays to store the values
    Arr1 = []
    Arr2 = []
    Arr3 = []

    for key, value in data.items():
        ### To read the file used
        if str('TRANSFORM').lower() in str(key).lower():
            lines = '\n'.join(value)
            lines = lines.split("\n")

            # Loop through the lines and extract the values
            for line in lines:
                line_values = line.split('=')  # split the line into values
                print(line_values)
                var_name = line_values[0].strip()  # extract the variable name

                var_value = line_values[1].strip().split()[0]  # extract the variable value


                # Regular expression pattern to extract the condition
                pattern = r'^\w+=\([^\)]*\)(?:\s+(.*?)(?:\n|$))?$'

                # Split the string into lines and extract the condition from each line
                conditions = []

                for line in line.split('\n'):
                    match = re.match(pattern, line)

                    if match:
                        condition = match.group(1)

                        if condition is not None:
                            conditions.append(condition.strip())
                        else:
                            conditions.append(None)

                # Append the values to the arrays
                Arr1.append(var_name)
                Arr2.append(var_value)

                Arr3.append(condition)

    return Arr1, Arr2, Arr3

### ---------------------------------------------------------------------------------------------------------
def EST_RSE_(data):
    for key, value in data.items():
        pass

df = FILTER_(data, df)
GROUP_(data, df)
TRANSFORM_(data)
RENAME_(data, df)
NEWVARIABLE_(data, df)
FILE_(data)



['AGE_GR', '("01") AGE<5']
['AGE_GR', '("02") AGE>', '5 && AGE<10']
['AGE_GR', '("03") AGE>', '10 && AGE<20']
['AGE_GR', '("04") AGE>', '20 && AGE<60']
['AGE_GR', '("05") AGE>', '60']
['YP', '(1)']
['YM', '(1)        SEX<>2']
['YM', '(0)\t\tOtherwise']
['YF', '(1)        SEX', '2']
['YF', '(0)\t\tOtherwise']
['YH', '(1)        GEDU>', '"11"']
['YH', '(0)\t\tOtherwise']
0         01010511
1         01010511
2         01010511
3         01010511
4         01010511
            ...   
576564    02363101
576565    02363101
576566    02363101
576567    02363101
576568    02363101
Name: STRMID, Length: 576569, dtype: object


In [6]:
def parse_query_string(query_string):
    # Replace curly quotes with straight quotes
    query_string = query_string.replace("”", "\"")

    if 'in' in query_string:
        if ('NOT in' or 'not in') in query_string:
            query_string = re.sub(r'(\b\w+\b)\s+NOT in\s*\(([^)]+)\)', r'~\1.isin([\2])', query_string)
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        elif 'in' in query_string:
            # Replace 'in' operators with 'isin' and add list brackets
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        else:
            pass

    else:
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)('[^']*'|"[^"]*")|(\b\w+\b)(\s*[<>=!]+\s*)("[^]*'|"[^"]*")''', r'\1\2\3', query_string)
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)(\d+)''', r'\1\2\3', query_string)

    # Replace double quotes with single quotes
    query_string = query_string.replace("\"", "'")

    # Replace 'AND' with '&', 'OR' with '|', and add parentheses
    query_string = query_string.replace(" AND ", " & ")\
                        .replace(" OR ", " | ").\
                            replace("&&", '&').\
                                replace('||', '|').\
                                    replace('<>', '!=').\
                                        replace('NOT', '~')

    # Replace column names with df[column] syntax
    query_string = re.sub(r'(\b\w+\b)=', r'\1==', query_string)


    # # Define a function to replace variable names with df["<variable_name>"]
    # def replace_var(match):
    #     return f'df["{match.group(0)}"]'

    # # Use regex to replace variable names with df["<variable_name>"]
    # pattern = re.compile(r'''\b\w+\b(?=\s*[<>=!]=*\s*[\'"\d])''')
    # new_string = pattern.sub(replace_var, query_string)

    query_string = re.sub(r"'(\d+)'|\"(\d+)\"", lambda match: str(int(match.group(1) or match.group(2))), query_string)
    query_string = re.sub(r'\[(\d+,\s*)+\d+\]', lambda match: '[' + match.group(0).replace("\'", "'") + ']', query_string)

    return query_string


final_parsed_query = []

for i in list(TRANSFORM_(data))[2]:
    # print(i)
    if i:
        parsed_query = parse_query_string(i)
        final_parsed_query.append(parsed_query)
    else:
        final_parsed_query.append(i)

### Calling the final_parsed_query function
### ---------------------------------------
final_parsed_query

['AGE_GR', '("01") AGE<5']
['AGE_GR', '("02") AGE>', '5 && AGE<10']
['AGE_GR', '("03") AGE>', '10 && AGE<20']
['AGE_GR', '("04") AGE>', '20 && AGE<60']
['AGE_GR', '("05") AGE>', '60']
['YP', '(1)']
['YM', '(1)        SEX<>2']
['YM', '(0)\t\tOtherwise']
['YF', '(1)        SEX', '2']
['YF', '(0)\t\tOtherwise']
['YH', '(1)        GEDU>', '"11"']
['YH', '(0)\t\tOtherwise']


['AGE<5',
 'AGE>=5 & AGE<10',
 'AGE>=10 & AGE<20',
 'AGE>=20 & AGE<60',
 'AGE>=60',
 None,
 'SEX!=2',
 'Otherwise',
 'SEX==2',
 'Otherwise',
 'GEDU>=11',
 'Otherwise']

In [7]:
Arr1 = list(TRANSFORM_(data))[0]
Arr2 = list(TRANSFORM_(data))[1]

# print(Arr2)
Arr2 = [i.strip('()\'\"') for i in Arr2]
Arr3 = final_parsed_query

Arr2 = [float(elem) if elem.replace('.', '', 1).isdigit() else elem for elem in Arr2]
print(Arr2)

['AGE_GR', '("01") AGE<5']
['AGE_GR', '("02") AGE>', '5 && AGE<10']
['AGE_GR', '("03") AGE>', '10 && AGE<20']
['AGE_GR', '("04") AGE>', '20 && AGE<60']
['AGE_GR', '("05") AGE>', '60']
['YP', '(1)']
['YM', '(1)        SEX<>2']
['YM', '(0)\t\tOtherwise']
['YF', '(1)        SEX', '2']
['YF', '(0)\t\tOtherwise']
['YH', '(1)        GEDU>', '"11"']
['YH', '(0)\t\tOtherwise']
['AGE_GR', '("01") AGE<5']
['AGE_GR', '("02") AGE>', '5 && AGE<10']
['AGE_GR', '("03") AGE>', '10 && AGE<20']
['AGE_GR', '("04") AGE>', '20 && AGE<60']
['AGE_GR', '("05") AGE>', '60']
['YP', '(1)']
['YM', '(1)        SEX<>2']
['YM', '(0)\t\tOtherwise']
['YF', '(1)        SEX', '2']
['YF', '(0)\t\tOtherwise']
['YH', '(1)        GEDU>', '"11"']
['YH', '(0)\t\tOtherwise']
[1.0, 2.0, 3.0, 4.0, 5.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]


In [8]:

# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)

new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

mismatched_names

set()

In [9]:
Arr3 = [i.replace("(['", "([").replace("'])", "])") if i is not None else 'True' if Arr3[j] == 'Otherwise' else True for j, i in enumerate(Arr3)]
Arr3

['AGE<5',
 'AGE>=5 & AGE<10',
 'AGE>=10 & AGE<20',
 'AGE>=20 & AGE<60',
 'AGE>=60',
 True,
 'SEX!=2',
 'Otherwise',
 'SEX==2',
 'Otherwise',
 'GEDU>=11',
 'Otherwise']

In [10]:
new_arr3 = []
for i, value in enumerate(Arr3):
    if value == 'Otherwise':
        other_values = set(['~(' + Arr3[j] + ')' for j in range(len(Arr3)) if Arr1[j] == Arr1[i] and Arr3[j] != 'Otherwise'])
        new_value = ' & '.join(other_values)
        print(new_value)
        new_arr3.append(new_value)
    else:
        new_arr3.append(value)
        print(value)

# new_arr3

AGE<5
AGE>=5 & AGE<10
AGE>=10 & AGE<20
AGE>=20 & AGE<60
AGE>=60
True
SEX!=2
~(SEX!=2)
SEX==2
~(SEX==2)
GEDU>=11
~(GEDU>=11)


In [11]:
Arr3 = new_arr3

for i in range(len(Arr2)):
    if type(Arr3[i]) == str:
        print(Arr3[i])

AGE<5
AGE>=5 & AGE<10
AGE>=10 & AGE<20
AGE>=20 & AGE<60
AGE>=60
SEX!=2
~(SEX!=2)
SEX==2
~(SEX==2)
GEDU>=11
~(GEDU>=11)


In [12]:
# Apply the queries in Arr3 to filter the rows of the dataframe
for i, query_str in enumerate(Arr3):

    if query_str is True:
        if type(Arr2[i]) == str:
            if '+' in Arr2[i]:
                df[Arr1[i]] = df[Arr2[i].split('+')].apply(lambda x: x.sum(), axis=1)
                print('*** First if', Arr2[i], Arr1[i], Arr3[i], i)

            else:
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                print('*** First else', Arr2[i], Arr1[i], Arr3[i], i)

    else:
        # try:
        # query_str = re.sub(r'0?\s?(\d+)', r'\1', query_str)
        print(query_str)
        df_query = df.query(query_str)
        # except:
        #     pass

        if type(Arr2[i]) == float:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('### Third if', Arr2[i], Arr1[i], Arr3[i])
        else:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('@@@ First else', Arr2[i], Arr1[i], Arr3[i])


        if type(Arr2[i]) == str:
            if Arr2[i] in list(df.columns):
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                # print('$$$ Fourth if', Arr2[i], Arr1[i], Arr3[i])

# print(df)

AGE<5
AGE>=5 & AGE<10
AGE>=10 & AGE<20
AGE>=20 & AGE<60
AGE>=60
SEX!=2
~(SEX!=2)
SEX==2
~(SEX==2)
GEDU>=11
~(GEDU>=11)


In [13]:
df.to_csv('WorkFile_PPSWR_Method_Example.csv')

In [14]:
# Get the current column names
old_col_names = df.columns.tolist()

# Create a dictionary with the new column names
new_col_names = {col: col.strip() for col in old_col_names}

# Rename the columns using the dictionary
df.rename(columns=new_col_names, inplace=True)

df['MULT'] = pd.to_numeric(df['MULT'])

# Calculate the SS_MULT and CMULT fields
df['SS_MULT'] = df['MULT'] / 100
df['CMULT'] = df['MULT'] / 100

lst_df = list(df.columns)

groupby_cols = GROUP_(data, df)

# Create a list of all possible subsets of groupby_cols
groupby_cols_subsets = [list(subset) for i in range(len(groupby_cols) + 1)
                        for subset in itertools.combinations(groupby_cols, i)]

# Select columns that are not in groupby_cols
cols = [col for col in df.columns if col in groupby_cols]

agg_dict = {col: sum for col in cols if (df[col].dtype == int or df[col].dtype == float) and col in groupby_cols}




**Step 1** ------------------------------------------------------------------------------------------------------------------------------------
*STEP-1: Generate Group level estimates Yhat & Xhat:
SELECT sec, age_gr, COUNT(*) as no_sam, SUM(yp*cmult) as yphat, SUM(ym*cmult) as ymhat,;
 SUM(yf*cmult) as yfhat,SUM(yh*cmult) as yhhat FROM wf_ppswr GROUP BY 1,2 into table W1

In [15]:

df['YP'] = pd.to_numeric(df['YP']) * df['CMULT']
df['YM'] = pd.to_numeric(df['YM']) * df['CMULT']
df['YF'] = pd.to_numeric(df['YF']) * df['CMULT']
df['YH'] = pd.to_numeric(df['YH']) * df['CMULT']
df['no_sam'] = df.shape[0]
w1 = df.groupby(['SEC', 'AGE_GR']).agg({'YP': 'sum', 'YM': 'sum', 'YF': 'sum', 'YH': 'sum', 'CMULT': 'sum'}).reset_index()
w1.rename(columns={'YP': 'yphat', 'YM': 'ymhat', 'YF': 'yfhat', 'YH': 'yhhat'}, inplace=True)

w1

,SEC,AGE_GR,yphat,ymhat,yfhat,yhhat,CMULT
0,1,1,5.829931e+07,2.944240e+07,2.885691e+07,0.00,5.829931e+07
1,1,2,8.047559e+07,4.250508e+07,3.797051e+07,0.00,8.047559e+07
2,1,3,1.731144e+08,9.499555e+07,7.811886e+07,16358976.65,1.731144e+08
3,1,4,4.300690e+08,2.162573e+08,2.138117e+08,81883483.19,4.300690e+08
4,1,5,7.431650e+07,3.800328e+07,3.631322e+07,2509864.76,7.431650e+07
5,2,1,2.155388e+07,1.164983e+07,9.904048e+06,0.00,2.155388e+07
6,2,2,2.819638e+07,1.541015e+07,1.278623e+07,0.00,2.819638e+07
7,2,3,6.589402e+07,3.642584e+07,2.946819e+07,10451790.30,6.589402e+07
8,2,4,2.088585e+08,1.076129e+08,1.012456e+08,93498401.48,2.088585e+08
9,2,5,3.207825e+07,1.583684e+07,1.624141e+07,7336130.81,3.207825e+07


In [16]:
w1.columns

Index(['SEC', 'AGE_GR', 'yphat', 'ymhat', 'yfhat', 'yhhat', 'CMULT'], dtype='object')

**STEP 2**  ------------------------------------------------------------------------------------------------

*Step-2: Generate STRMID (X SSS) level Estimates of each variable for the same combination

SELECT sec, age_gr, strmid,CapZs,SmlZs,COUNT(dist(FSU)) as nd_fsu, SUM(yp*cmult) as ypstr, SUM(ym*cmult) as ymstr,;
 SUM(yf*cmult) as yfstr,SUM(yh*cmult) as yhstr FROM wf_ppswr GROUP BY 1,2,3,4,5 into table W2


In [17]:

# df['ypstr'] = pd.to_numeric(df['YP']) * df['CMULT']
# df['ymstr'] = pd.to_numeric(df['YM']) * df['CMULT']
# df['yfstr'] = pd.to_numeric(df['YF']) * df['CMULT']
# df['yhstr'] = pd.to_numeric(df['YH']) * df['CMULT']


# w2 = df.groupby(['SEC', 'AGE_GR', 'STRMID']).agg(
#                                                     no_sam=('SEC', 'count'),
#                                                     ypstr=('ypstr', 'sum'),
#                                                     ymstr=('ymstr', 'sum'),
#                                                     yfstr=('yfstr', 'sum'),
#                                                     yhstr=('yhstr', 'sum'),
#                                                     ndfsu = ('FSU', 'nunique')
#                                                 ).reset_index()

df['ndfsu'] = df['FSU'] + df['SSS']

w2 = df.groupby(['SEC', 'AGE_GR', 'STRMID', 'SSS']).agg(
    ndfsu=('ndfsu', pd.Series.nunique),
    ypstr=('YP', 'sum'),
    ymstr=('YM', 'sum'),
    yfstr=('YF', 'sum'),
    yhstr=('YH', 'sum')
).reset_index()


w2

,SEC,AGE_GR,STRMID,SSS,ndfsu,ypstr,ymstr,yfstr,yhstr
0,1,1,01010101,1,1,71.54,35.77,35.77,0.0
1,1,1,01010101,6,1,214.62,35.77,178.85,0.0
2,1,1,01010101,7,1,4793.40,0.00,4793.40,0.0
3,1,1,01010111,7,1,40277.25,40277.25,0.00,0.0
4,1,1,01010201,1,1,70.34,70.34,0.00,0.0
...,...,...,...,...,...,...,...,...,...
96697,2,5,02363222,6,2,1490.00,894.00,596.00,1192.0
96698,2,5,02363223,2,1,269.00,269.00,0.00,0.0
96699,2,5,02363223,4,1,807.00,538.00,269.00,0.0
96700,2,5,02363223,6,1,538.00,269.00,269.00,0.0


In [18]:
df.columns

Index(['SEC', 'ST', 'AST', 'DC', 'NSSR', 'STRM', 'SSTRM', 'FSU', 'SSS', 'SSU',
       'CAPZS', 'SMLZS', 'DSI', 'NFSU', 'NSS', 'MULT', 'CAPH', 'SMLH', 'SRL',
       'HHSIZE', 'RELIGION', 'SG', 'MPCE_ALL', 'MPCE_ST', 'REL', 'SEX', 'AGE',
       'MARST', 'GEDU', 'TEDU', 'AGE_GR', 'STRMID', 'YP', 'YM', 'YF', 'YH',
       'SS_MULT', 'CMULT', 'no_sam', 'ndfsu'],
      dtype='object')

**STEP 3**
-----------------------------------------------------------------------------
*Step-3. Generate STRMID (X SSS) X FSU level Estimates of each variable for the same combination
SELECT sec, age_gr, strmid,FSU,nfsu as ns, SUM(yp*cmult) as ypfsu, SUM(ym*cmult) as ymfsu,;
 SUM(yf*cmult) as yffsu,SUM(yh*cmult) as yhfsu FROM wf_ppswr GROUP BY 1,2,3,4,5 into table W3


In [19]:
# df['ypfsu'] = pd.to_numeric(df['YP']) * df['CMULT']
# df['ymfsu'] = pd.to_numeric(df['YM']) * df['CMULT']
# df['yffsu'] = pd.to_numeric(df['YF']) * df['CMULT']
# df['yhfsu'] = pd.to_numeric(df['YH']) * df['CMULT']
df['ns'] = df['NSS']

w3 = df.groupby(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'ns', 'FSU']).agg(
                                                                ypfsu=('YP', 'sum'),
                                                                    ymfsu=('YM', 'sum'),
                                                                yffsu=('YF', 'sum'),
                                                                    yhfsu=('YH', 'sum')
                                                                ).reset_index()


# w2 = df.groupby(['SEC', 'AGE_GR', 'STRMID']).agg(
#     ndfsu=('FSU', pd.Series.nunique),
#     ypstr=('YP', 'sum'),
#     ymstr=('YM', 'sum'),
#     yfstr=('YF', 'sum'),
#     yhstr=('YH', 'sum')
# ).reset_index()

w3

,SEC,AGE_GR,STRMID,SSS,ns,FSU,ypfsu,ymfsu,yffsu,yhfsu
0,1,1,01010101,1,2.0,10066,71.54,35.77,35.77,0.0
1,1,1,01010101,6,2.0,10066,214.62,35.77,178.85,0.0
2,1,1,01010101,7,2.0,10066,4793.40,0.00,4793.40,0.0
3,1,1,01010111,7,2.0,10022,40277.25,40277.25,0.00,0.0
4,1,1,01010201,1,2.0,10024,70.34,70.34,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...
139840,2,5,02363223,2,2.0,27104,269.00,269.00,0.00,0.0
139841,2,5,02363223,4,2.0,27046,807.00,538.00,269.00,0.0
139842,2,5,02363223,6,2.0,27046,538.00,269.00,269.00,0.0
139843,2,5,02363223,7,2.0,27046,34835.50,34835.50,0.00,0.0


--------------------


**STEP - 4**: Combine all three estimates at STRMID (X SSS) level for the same combination
SELECT a.*,b.CapZs,b.SmlZs,b.nd_fsu,b.ypstr,b.ymstr,b.yfstr,b.yhstr;
from w3 a, w2 b WHERE a.sec+a.age_gr+a.strmid = b.sec+b.age_gr+b.strmid INTO TABLE **W4**

In [20]:
w4 = pd.merge(w3, w2[['SEC', 'AGE_GR','SSS', 'STRMID', 'ndfsu', 'ypstr', 'ymstr', 'yfstr', 'yhstr']], on=['SEC', 'AGE_GR', 'STRMID', 'SSS'], how='inner')
w4

,SEC,AGE_GR,STRMID,SSS,ns,FSU,ypfsu,ymfsu,yffsu,yhfsu,ndfsu,ypstr,ymstr,yfstr,yhstr
0,1,1,01010101,1,2.0,10066,71.54,35.77,35.77,0.0,1,71.54,35.77,35.77,0.0
1,1,1,01010101,6,2.0,10066,214.62,35.77,178.85,0.0,1,214.62,35.77,178.85,0.0
2,1,1,01010101,7,2.0,10066,4793.40,0.00,4793.40,0.0,1,4793.40,0.00,4793.40,0.0
3,1,1,01010111,7,2.0,10022,40277.25,40277.25,0.00,0.0,1,40277.25,40277.25,0.00,0.0
4,1,1,01010201,1,2.0,10024,70.34,70.34,0.00,0.0,1,70.34,70.34,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139840,2,5,02363223,2,2.0,27104,269.00,269.00,0.00,0.0,1,269.00,269.00,0.00,0.0
139841,2,5,02363223,4,2.0,27046,807.00,538.00,269.00,0.0,1,807.00,538.00,269.00,0.0
139842,2,5,02363223,6,2.0,27046,538.00,269.00,269.00,0.0,1,538.00,269.00,269.00,0.0
139843,2,5,02363223,7,2.0,27046,34835.50,34835.50,0.00,0.0,2,58238.50,34835.50,23403.00,0.0


*Combining YHAT & XHAT
SELECT a.*,b.yphat,b.ymhat,b.yfhat,b.yhhat, b.yfhat/ymhat as r1hat,b.yhhat/yphat as r2hat;
from w4 a, w1 b WHERE a.sec+a.age_gr = b.sec+b.age_gr INTO TABLE W5

In [21]:
w1['r1hat'] = w1['yfhat'] / w1['ymhat']
w1['r2hat'] = w1['yhhat'] / w1['yphat']
w5 = pd.merge(w4, w1[['SEC', 'AGE_GR', 'yphat', 'ymhat', 'yfhat', 'yhhat', 'r1hat', 'r2hat']], on=['SEC', 'AGE_GR'], how='inner')


In [22]:
w5

,SEC,AGE_GR,STRMID,SSS,ns,FSU,ypfsu,ymfsu,yffsu,yhfsu,...,ypstr,ymstr,yfstr,yhstr,yphat,ymhat,yfhat,yhhat,r1hat,r2hat
0,1,1,01010101,1,2.0,10066,71.54,35.77,35.77,0.0,...,71.54,35.77,35.77,0.0,58299310.61,29442398.25,28856912.36,0.00,0.980114,0.000000
1,1,1,01010101,6,2.0,10066,214.62,35.77,178.85,0.0,...,214.62,35.77,178.85,0.0,58299310.61,29442398.25,28856912.36,0.00,0.980114,0.000000
2,1,1,01010101,7,2.0,10066,4793.40,0.00,4793.40,0.0,...,4793.40,0.00,4793.40,0.0,58299310.61,29442398.25,28856912.36,0.00,0.980114,0.000000
3,1,1,01010111,7,2.0,10022,40277.25,40277.25,0.00,0.0,...,40277.25,40277.25,0.00,0.0,58299310.61,29442398.25,28856912.36,0.00,0.980114,0.000000
4,1,1,01010201,1,2.0,10024,70.34,70.34,0.00,0.0,...,70.34,70.34,0.00,0.0,58299310.61,29442398.25,28856912.36,0.00,0.980114,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139840,2,5,02363223,2,2.0,27104,269.00,269.00,0.00,0.0,...,269.00,269.00,0.00,0.0,32078251.56,15836843.26,16241408.30,7336130.81,1.025546,0.228695
139841,2,5,02363223,4,2.0,27046,807.00,538.00,269.00,0.0,...,807.00,538.00,269.00,0.0,32078251.56,15836843.26,16241408.30,7336130.81,1.025546,0.228695
139842,2,5,02363223,6,2.0,27046,538.00,269.00,269.00,0.0,...,538.00,269.00,269.00,0.0,32078251.56,15836843.26,16241408.30,7336130.81,1.025546,0.228695
139843,2,5,02363223,7,2.0,27046,34835.50,34835.50,0.00,0.0,...,58238.50,34835.50,23403.00,0.0,32078251.56,15836843.26,16241408.30,7336130.81,1.025546,0.228695


***Step-5**: Calculate variance of variables Y, X & R at STRMID level
SELECT sec,age_gr,strmid,ns,nd_fsu,SUM(((ns*(yffsu - r1hat*ymfsu)) - (yfstr - r1hat*ymstr))^2) as myfym,;
SUM((ns*yhfsu - yhstr)^2) as vyh,;
SUM(((ns*(yhfsu - r2hat*ypfsu)) - (yhstr - r2hat*ypstr))^2) as myhyp;
FROM w5 GROUP BY 1,2,3,4,5 INTO TABLE w6

In [23]:
w6 = w5.groupby(['SEC', 'AGE_GR', 'STRMID', 'ns', 'ndfsu', 'SSS']).agg(
    myfym=('ns', lambda x: np.sum(((x * (w5['yffsu'] - w5['r1hat'] * w5['ymfsu'])) - (w5['yfstr'] - w5['r1hat'] * w5['ymstr'])) ** 2)),
    vyh=('ns', lambda x: np.sum((x * w5['yhfsu'] - w5['yhstr']) ** 2)),
    myhyp=('ns', lambda x: np.sum(((x * (w5['yhfsu'] - w5['r2hat'] * w5['ypfsu'])) - (w5['yhstr'] - w5['r2hat'] * w5['ypstr'])) ** 2))
).reset_index()
w6

,SEC,AGE_GR,STRMID,ns,ndfsu,SSS,myfym,vyh,myhyp
0,1,1,01010101,2.0,1,1,5.059695e-01,0.0,0.000000e+00
1,1,1,01010101,2.0,1,6,2.067594e+04,0.0,0.000000e+00
2,1,1,01010101,2.0,1,7,2.297668e+07,0.0,0.000000e+00
3,1,1,01010111,2.0,1,7,1.558379e+09,0.0,0.000000e+00
4,1,1,01010201,2.0,1,1,4.752894e+03,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...
96697,2,5,02363222,2.0,2,6,1.867982e+05,0.0,9.289135e+03
96698,2,5,02363223,2.0,1,2,7.610526e+04,0.0,3.784577e+03
96699,2,5,02363223,2.0,1,4,7.994397e+04,0.0,3.406119e+04
96700,2,5,02363223,2.0,1,6,4.722196e+01,0.0,1.513831e+04


** Generating variance for ZERO cases

** SELECT sec,age_gr,strmid,ns,nd_fsu,SUM((ns-nd_fsu)*(0 - (yfstr - r1hat*ymstr))^2) as myfym,;
SUM((ns-nd_fsu)*(0 - yhstr)^2) as vyh,;
SUM((ns-nd_fsu)*(0 - (yhstr - r2hat*ypstr))^2) as myhyp;
FROM w5 GROUP BY 1,2,3,4,5 INTO TABLE w7 **

In [ ]:
w7 = w5.groupby(['SEC', 'AGE_GR', 'STRMID', 'ns', 'ndfsu', 'SSS']).agg(
    myfym=('ns', lambda x: np.sum(((x - w5['ndfsu']) * (0 - (w5['yfstr'] - w5['r1hat'] * w5['ymstr']))) ** 2)),
    vyh=('ns', lambda x: np.sum(((x - w5['ndfsu']) * (0 - w5['yhstr'])) ** 2)),
    myhyp=('ns', lambda x: np.sum(((x - w5['ndfsu']) * (0 - (w5['yhstr'] - w5['r2hat'] * w5['ypstr']))) ** 2))
).reset_index()
w7

In [ ]:
w7.columns

** Adding both the variances

UPDATE w6 from w7 ;
SET w6.myfym = w6.myfym + w7.myfym ,;
 w6.vyh = w6.vyh + w7.vyh ,;
 w6.myhyp = w6.myhyp + w7.myhyp ;
WHERE w6.sec+w6.age_gr+w6.strmid = w7.sec+w7.age_gr+w7.strmid

In [ ]:
w6['myfym'] = w6['myfym'] + w7['myfym']
w6['vyh'] = w6['vyh'] + w7['vyh']
w6['myhyp'] = w6['myhyp'] + w7['myhyp']


w6 = w6.merge(w7, on=['SEC', 'AGE_GR', 'STRMID', 'SSS'], suffixes=('', '_w7'))
w6['myfym'] = w6['myfym'] + w6['myfym_w7']
w6['vyh'] = w6['vyh'] + w6['vyh_w7']
w6['myhyp'] = w6['myhyp'] + w6['myhyp_w7']
w6.drop(['myfym_w7', 'vyh_w7', 'myhyp_w7'], axis=1, inplace=True)
w6

** Divide by ns*(ns-1)

CLOSE ALL
USE w6
REPLACE vyh WITH vyh/(ns*ns-1) FOR ns>1
REPLACE myfym WITH myfym/(ns*ns-1) FOR ns>1
REPLACE myhyp WITH myhyp/(ns*ns-1) FOR ns>1

In [ ]:
import numpy as np

# # Replace the variables with the updated values
# w6.loc[w6['ns'] > 1, 'vyh'] = w6.loc[w6['ns'] > 1, 'vyh'] / (w6.loc[w6['ns'] > 1, 'ns'] * (w6.loc[w6['ns'] > 1, 'ns'] - 1))
# w6.loc[w6['ns'] > 1, 'myfym'] = w6.loc[w6['ns'] > 1, 'myfym'] / (w6.loc[w6['ns'] > 1, 'ns'] * (w6.loc[w6['ns'] > 1, 'ns'] - 1))
# w6.loc[w6['ns'] > 1, 'myhyp'] = w6.loc[w6['ns'] > 1, 'myhyp'] / (w6.loc[w6['ns'] > 1, 'ns'] * (w6.loc[w6['ns'] > 1, 'ns'] - 1))

# # Replace NaN values resulting from divisions by 0 with 0
# w6 = w6.replace(np.nan, 0)
# w6

condition = w6['ns'] > 1

# Perform replacements and divisions
w6['vyh'] = w6['vyh'].where(condition, w6['vyh'] / (w6['ns'] * (w6['ns'] - 1)))
w6['myfym'] = w6['myfym'].where(condition, w6['myfym'] / (w6['ns'] * (w6['ns'] - 1)))
w6['myhyp'] = w6['myhyp'].where(condition, w6['myhyp'] / (w6['ns'] * (w6['ns'] - 1)))

w6

** Generating Variance at Group level

SELECT sec,age_gr,SUM(vyh) as vyh,SUM(myfym) as myfym, SUM(myhyp) as myhyp ;
FROM w6 GROUP BY 1,2 INTO TABLE w8

In [ ]:
w8 = w6.groupby(['SEC', 'AGE_GR']).agg({'vyh': 'sum', 'myfym': 'sum', 'myhyp': 'sum'}).reset_index()
w8.columns = ['SEC', 'AGE_GR', 'vyh', 'myfym', 'myhyp']
w8

** Post related estimates from W1

SELECT a.*,b.no_sam,b.yphat,b.ymhat,b.yfhat,b.yhhat, b.yfhat/ymhat as r1hat,b.yhhat/yphat as r2hat;
from w8 a, w1 b WHERE a.sec+a.age_gr = b.sec+b.age_gr INTO TABLE W9

In [ ]:
w1['r1hat'] = w1['yfhat'] / w1['ymhat']
w1['r2hat'] = w1['yhhat'] / w1['yphat']
w9 = pd.merge(w8, w1, on=['SEC', 'AGE_GR'])

w9

* Calculate final Estimates and RSE

SELECT sec,SPACE(12) as sec_desc,age_gr,SPACE(12) as agegr_desc,no_sam,1000*R1hat as R1Hat, ;
IIF(YFHat>0, 100*(myfym^0.5)/YFHat, 0.00) as RSE_YfYm, ;
IIF(YHHat>0, 100*(vyh^0.5)/YHHat,0.00) as RSE_YH,;
100*R2Hat as R2Hat,;
IIF(YHHat>0, 100*(myhyp^0.5)/YHHat, 0.00) as RSE_YhYp ;
FROM W9 INTO TABLE T1

In [ ]:
T1 = w9.copy()
T1['sec_desc'] = ''
T1['agegr_desc'] = ''
T1['R1Hat'] = 1000 * T1['r1hat']
T1['RSE_YfYm'] = np.where(T1['yfhat'] > 0, 100 * (T1['myfym'] ** 0.5) / T1['yfhat'], 0.00)
T1['RSE_YH'] = np.where(T1['yhhat'] > 0, 100 * (T1['vyh'] ** 0.5) / T1['yhhat'], 0.00)
T1['R2Hat'] = 100 * T1['r2hat']
T1['RSE_YhYp'] = np.where(T1['yhhat'] > 0, 100 * (T1['myhyp'] ** 0.5) / T1['yhhat'], 0.00)


In [ ]:
T1 = T1[['SEC', 'AGE_GR', 'sec_desc', 'agegr_desc', 'R1Hat', 'RSE_YfYm', 'RSE_YH', 'R2Hat', 'RSE_YhYp']]
T1

In [ ]:
# # Creating T2
# T2 = T1.copy()
# T2['Group'] = 'Rural, 0-5'
# T2 = T2[(T2['SEC'] == '1') & (T2['AGE_GR'] == '01')]

# # Creating T3
# T3 = T1.copy()
# T3['Group'] = 'Urban, 0-5'
# T3 = T3[(T3['SEC'] == '2') & (T3['AGE_GR'] == '01')]

# # Creating T4
# T4 = T1.copy()
# T4['Group'] = 'Rural+Urban, 0-5'
# T4 = T4[(T4['SEC'] == 'Z') & (T4['AGE_GR'] == '01')]

# # Merging T2, T3, and T4 together
# merged_df = pd.concat([T2, T3, T4], ignore_index=True)

# # Updating SEC and AGE_GR descriptions
# merged_df['sec_desc'] = merged_df['SEC'].replace({'1': 'Rural', '2': 'Urban', 'Z': 'Rural+Urban'})
# merged_df['agegr_desc'] = merged_df['AGE_GR'].replace({'01': '0 - 5', '02': '5 - 10', '03': '10 - 20', '04': '20 - 60', '05': '>= 60'})


In [ ]:
# merged_df